In [1]:
import pandas as pd


In [9]:
data = pd.read_csv("plot.txt", header=None, delimiter="\n")

In [10]:
data.head()


,0
0,the movie begins in the past where a young boy...
1,emerging from the human psyche and showing cha...
2,spurning her mother's insistence that she get ...
3,amitabh can't believe the board of directors a...
4,"she , among others excentricities , talks to a..."


In [12]:
data[0][0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . '

In [14]:
import nltk.data # библиотека Natural Language Toolkit
import re   # библиотека для регулярных выражений
from nltk.corpus import stopwords # стоп-слова из NLTK
nltk.download('punkt') # пунктуация для правильной работы токенизатора

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luigi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


In [22]:
def review_to_wordlist(review, remove_stopwords=False):
    # оставляем только буквенные символы
    review = re.sub("[^a-zA-Z]"," ", review)
    # разбиваем на слова по символу пробела
    words = review.split()
    if remove_stopwords:
      # убираем стоп-слова
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return words

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sent_list = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sent_list.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sent_list

In [23]:
sentences = []  

print("Parsing sentences from training set...")
for sent in data[0]:
    sentences += review_to_sentences(sent, tokenizer)
    

Parsing sentences from training set...


In [24]:
len(sentences)


5553

In [42]:
sentences[4740]


['adapted',
 'from',
 'a',
 'monologue',
 'in',
 'his',
 'blue',
 'jam',
 'radio',
 'series',
 'chris',
 'morris',
 'first',
 'short',
 'film',
 'is',
 'a',
 'haunting',
 'black',
 'comedy',
 'about',
 'a',
 'man',
 'who',
 'no',
 'longer',
 'uses',
 'his',
 'name',
 'because',
 'he',
 's',
 'decided',
 'he',
 's',
 'ceased',
 'to',
 'deserve',
 'one',
 'and',
 'a',
 'dog',
 'called',
 'rothko',
 'who',
 'says',
 'he',
 'is',
 'the',
 'man',
 's',
 'lawyer']

In [34]:
import gensim 
from gensim.models import word2vec
from gensim.models import Word2Vec

In [91]:
print("Training model...")
%time model_en = word2vec.Word2Vec(sentences, vector_size=100, window=10, min_count=5, workers=4)

Training model...
Wall time: 551 ms


In [92]:
print(len(model_en.wv.index_to_key))


2901


In [93]:
print(model_en.wv.most_similar("plot", topn=3))
print(model_en.wv.most_similar("comedy", topn=3))
print(model_en.wv.most_similar("drama", topn=3))
print(model_en.wv.most_similar("thriller", topn=3))
print(model_en.wv.most_similar("movie", topn=3))

[('personal', 0.9987538456916809), ('through', 0.9987449049949646), ('power', 0.9987229704856873)]
[('american', 0.9995850920677185), ('dark', 0.999584972858429), ('documentary', 0.9995837211608887)]
[('american', 0.9993696808815002), ('movie', 0.9993507266044617), ('this', 0.9993354678153992)]
[('school', 0.9983227849006653), ('long', 0.9982951879501343), ('police', 0.9982432126998901)]
[('american', 0.9997138977050781), ('town', 0.9997057914733887), ('between', 0.9996850490570068)]


In [138]:
print(model_en.wv.most_similar(positive=["fan", "movie"], topn=1)[0][0])

people


In [139]:
print(model_en.wv.most_similar(positive=["drama", "love"], topn=1)[0][0])


girl


In [148]:
model_en.wv.similarity('police', 'murder')

0.999546

In [160]:
model_en.wv.similarity('horse', 'monster')

0.9845945

## Дообучим модель на втором корпусе

In [161]:
data_2 = pd.read_csv("quote.txt", header=None, delimiter="\n")


In [162]:
data_2.head()


,0
0,"smart and alert , thirteen conversations about..."
1,"color , musical bounce and warm seas lapping o..."
2,it is not a mass-market entertainment but an u...
3,a light-hearted french film about the spiritua...
4,my wife is an actress has its moments in looki...


In [163]:
new_sentences = []  

print("Parsing sentences from training set...")
for sent in data_2[0]:
    new_sentences += review_to_sentences(sent, tokenizer)


Parsing sentences from training set...


In [164]:
model_path = "movie_reviews.model"
print("Saving model...")
model_en.save(model_path)


Saving model...


In [165]:
model = word2vec.Word2Vec.load(model_path)


In [166]:
model.build_vocab(new_sentences, update=True)
model.train(new_sentences, total_examples=model.corpus_count, epochs=5)


(319641, 526650)

In [167]:
print(model_en.wv.most_similar("plot", topn=3))
print(model_en.wv.most_similar("comedy", topn=3))
print(model_en.wv.most_similar("drama", topn=3))
print(model_en.wv.most_similar("thriller", topn=3))
print(model_en.wv.most_similar("movie", topn=3))


[('personal', 0.9987538456916809), ('through', 0.9987449049949646), ('power', 0.9987229704856873)]
[('american', 0.9995850920677185), ('dark', 0.999584972858429), ('documentary', 0.9995837211608887)]
[('american', 0.9993696808815002), ('movie', 0.9993507266044617), ('this', 0.9993354678153992)]
[('school', 0.9983227849006653), ('long', 0.9982951879501343), ('police', 0.9982432126998901)]
[('american', 0.9997138977050781), ('town', 0.9997057914733887), ('between', 0.9996850490570068)]


In [168]:
print(model_en.wv.most_similar(positive=["fan", "movie"], topn=1)[0][0])
print(model_en.wv.most_similar(positive=["drama", "love"], topn=1)[0][0])

people
girl


In [169]:
model_en.wv.similarity('police', 'murder')

0.999546

In [170]:
model_en.wv.similarity('horse', 'monster')


0.9845945